## Install Libraries


In [1]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Import Libraries


In [2]:
from collections import OrderedDict
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [3]:
ray.init(ignore_reinit_error=True, num_cpus=4, num_gpus=4)
print("success")


2023-04-08 02:15:12,252	INFO worker.py:1553 -- Started a local Ray instance.


success


## Data loaders


In [4]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [5]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [6]:
# Will need to modify the parameters and the network depending on what you are experimenting with
class Net(nn.Module):
    def __init__(self, kernel_size):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(3, 3, kernel_size, padding="same")
        self.fc = nn.Linear(3 * 32 * 32, 10)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = flatten(x)
        x = self.fc(x)
        return x


## The train function


In [7]:
def train_cifar(config):
    net = Net(
        config["kernel_size"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-3)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [8]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["kernel_size"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)

    new_model_state = OrderedDict()
    for key, value in model_state.items():
        name = key[7:]  # remove `module.`
        new_model_state[name] = value

    best_trained_model.load_state_dict(new_model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [9]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"kernel_size": tune.grid_search([1, 2, 3, 5, 10, 15, 32])}


In [10]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"cpu": 4, "gpu": 4},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


0it [00:00, ?it/s]far pid=1801667) 
  0%|          | 8192/170498071 [00:00<41:15, 68864.23it/s]


(train_cifar pid=1801667) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /common/home/vig4/ray_results/train_cifar_2023-04-08_02-15-13/train_cifar_b8ee4_00000_0_kernel_size=1_2023-04-08_02-15-13/data/cifar-10-python.tar.gz


 99%|█████████▉| 168828928/170498071 [00:12<00:00, 13343691.10it/s]


(train_cifar pid=1801667) Files already downloaded and verified


(train_cifar pid=1801667) /koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
(train_cifar pid=1801667)   img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
(train_cifar pid=1801667) /koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
(train_cifar pid=1801667)   img = torch.ByteTensor(torch.ByteStorage.fr

...

(train_cifar pid=1801667) Files already downloaded and verified


170500096it [00:06, 25075919.31it/s]                               /koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
(train_cifar pid=1801667)   img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
(train_cifar pid=1801667) /koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
(train_cifar pid=1801667)   im

Best trial config: {'kernel_size': 32}
Best trial final validation loss: 1.629443556639799
Best trial final validation accuracy: 0.4252
(train_cifar pid=1801667) Finished Training
Files already downloaded and verified
Files already downloaded and verified


/koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
/koko/system/anaconda/envs/python39/lib/python3.9/site-packages/torchvision/transforms/functional.py:79: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
/common/home/vig4/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:

Best trial test set accuracy: 0.4379


In [11]:
results_df.drop(
    columns=[
        "should_checkpoint",
        "done",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)


,loss,accuracy,time_this_iter_s,time_total_s,config/kernel_size
0,2.030164,0.3049,5.525124,89.053396,1
1,1.911807,0.3401,6.023555,82.091532,2
2,1.812696,0.3670,5.822653,62.679997,3
3,1.734980,0.4027,6.120526,64.585817,5
4,1.718148,0.3955,5.808425,63.341961,10
5,1.648080,0.4178,6.327283,64.897759,15
6,1.629444,0.4252,5.653556,69.697712,32
